ML End Module Exam

Pandas Data Cleaning-
One dataframe given,
Ek column leke standard scalar kaise karna hai, get dummies kaise karna hai
Ek que data pe rahega jispe classification algo use karna hai
Preferred- use grid search series 

Regression ka ek problem
You can do regression using Neural Network 
SImple neural network dense layer lagake
Jitne bhi dense layer banaoge, output layer me 1 neuron lena hai aur 
Loss should be mean squared error, mae, model.compile

Day 13 keras simple neural .ipynb


In [ ]:
from sklearn.cluster import KMeans
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np

# Agglomerative clustering

In [ ]:
import matplotlib.pyplot as plt

# Draw Dendogram
import scipy.cluster.hierarchy as shc
plt.figure(figsize=(10, 7))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(X, method='ward'))
plt.axhline(y=50, color='r', linestyle='--')


# Agglomaerative clustering
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering( n_clusters= 3 , affinity='euclidean', linkage='ward')  
cluster_labels = cluster.fit_predict(X)

# affinity is how you calculate dist between two points.
# linkage is how you find closeness between two clusters.

# plot
plt.scatter(X[:,0], X[:,1], c=cluster_labels, cmap='rainbow')

# DBSCAN

In [ ]:
# Knee Finding in DBSCAN

from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(X)
distances, indices = nbrs.kneighbors(X) #
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)

# till here you will get best eps
#----------------------------------------------------------------------------

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X) # standardization

# before finding the value of eps data must be in """"""standardized form"""""".
# transform() is used to transform the data
# fit_transform() - train the model and also transform it afterwards.


y_pred = DBSCAN(eps=0.3, min_samples=10).fit_predict(X)
# fit_predict(), train
# DBSCAN doesnt have predict function, since it can not predict for test data(new data).
#you have to fit the new data to know the outliers

import numpy as np
# find the number of clusters
print('Number of clusters: {}'.format(len(set(y_pred[np.where(y_pred != -1)]))))

print('Outliers :', len(y_pred[np.where(y_pred == -1)]))
# -1 means outliers

# plot
plt.scatter(X[:,0], X[:,1], c=y_pred, cmap='rainbow')

# KMeans

In [ ]:
import pandas as pd

df = pd.read_csv("diamonds.csv") #explore df

# to remove unnamed:0 column use index_col = 0 inside read_csv
df = pd.read_csv("diamonds.csv",index_col =0) # explore df

df_ohe = pd.get_dummies(df) #explore df_ohe

# training and testing split
from sklearn.model_selection import train_test_split
X = df_ohe

# we only have X and not y(target variable)
X_train,X_test = train_test_split(X,test_size = 0.3, random_state = 7)
X_train.shape, X_test.shape

#----------------------------------------------------------------------------
# try various values of k from 2 to 10
inertia_list = []
for k in range(2,11):
    km = KMeans(n_clusters = k,random_state = 7)
    km.fit(X_train)
    clust_nos = km.predict(X_test)
    
    inertia_list.append(km.inertia_)
# Elbow graph
from matplotlib import pyplot as plt
plt.plot(range(2,11),inertia_list)
# if you see a great bend at any value then that k value is best


# check which k value is best
s_score_dict = {}
for k in range(2,11):
    km = KMeans(n_clusters = k,random_state = 7)
    km.fit(X_train)
    clust_nos = km.predict(X_test)

    s = silhouette_score(X_test,clust_nos,random_state = 7)
    s_score_dict[k] = s
print(s_score_dict)
#----------------------------------------------------------------------------



# Perfom KMeans clustering
from sklearn.cluster import KMeans

# Create object of the model
km = KMeans(n_clusters = 3,random_state = 7)
# Train the model using fit()
km.fit(X_train)
km.cluster_centers_# 3 centroids and each centroid will have 27 dimensions(columns).

# Get the predictions from the model using predict
clust_nos = km.predict(X_test) # final prediction of classes

# silhouette score
from sklearn.metrics import silhouette_score
silhouette_score(X_test,clust_nos,random_state = 7)

#----------------------------------------------------------------------------
# plot
plt.scatter(X[:, 0], X[:, 1], c=clust_nos, s=50, cmap='viridis')
centers = km.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

# Outlier Handling

In [ ]:
num_columns = df.select_dtypes(exclude='object').columns

## Z-score based / Standard Scaler based method
# -----------------------
def zscore_for_column(df,col):
    #zscore = (val - mean) / std_dev
    zscore = (df[col] - df[col].mean())/df[col].std()
    return zscore


# calculating zscore of each column and updating it in the copy.
df_std = df_ohe.copy()
for col in num_columns:
    df_std[col] = zscore_for_column(df,col)
    
df_std.describe()


# OR

# Z score / Standardscaler using library
df_std =df_ohe.copy()
from sklearn.preprocessing import StandardScaler
sd = StandardScaler()
sd.fit(df_std.loc[:,num_columns]) # it will learn --> mean and std_dev

# acually apply the formul and convert the data --> transform
df_std.loc[:,num_columns]=sd.transform(df_std.loc[:,num_columns])

df_std.describe()

# ---------------------------------

# Replacing the outliers with -3 and 3 (extreme values)

from IPython.utils.text import columnize
def outlier_imputation(df,col):
    print("number of outliers in ",col,end=" ")
    print((df.loc[df[col]< -3,col]).count()+(df.loc[df[col] > 3 ,col]).count())
    #print(df.loc[(df[col] < -3) | (df[col] > +3), :].shape[0])
    df.loc[df[col] < -3 ,col] = -3 
    df.loc[df[col] > 3 ,col] = 3
    return df


from IPython.utils.text import columnize
def outlier_imputation(df,col):
    print("No of outliers in ", col , " are" , df.loc[(df[col] < -3) | (df[col] > +3), :].shape[0])
    df.loc[df[col] < -3 ,col] = -3
    df.loc[df[col] > 3 ,col] = 3
    return df

for col in num_columns:
    df_std = outlier_imputation(df_std,col)
    
    
# --------------------------------------------

########################################################################
# IQR Based outlier handling
df_iqr = df_ohe.copy()
q1, q3 = df['table'].quantile([0.25,0.75])
def outlier_imputation_IQR(df,col):
    q1, q3 = df[col].quantile([0.25,0.75])
    iqr = q3 -q1

    df.loc[df[col] < (q1-1.5*iqr),col ] = (q1-1.5*iqr) 
    df.loc[df[col] > (q3+1.5*iqr),col ] = (q3+1.5*iqr)
    return df

for col in num_columns:
    df_iqr = outlier_imputation_IQR(df_iqr,col)

df_iqr.describe()

########################################################################


# GridSearch

## Ridge Regression

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0.01,0.5,1,5],
    'max_iter':[100,500,1000,5000]
}

# Ridge regression using grid search
gscv = GridSearchCV(Ridge(random_state=7),param_grid,
                scoring='r2',cv=2,verbose=2)

gscv.fit(X_train,Y_train)

print(gscv.best_score_)

print(gscv.best_params_)

# Create model with best parameters
rg = Ridge(alpha=0.01, max_iter=100, random_state=7)
Y_pred= rg.predict(X_test)

from sklearn.metrics import r2_score

print(r2_score(Y_test,Y_pred))

# Predict on new data
X_test.iloc[0:1,:]
log_price = rg.predict(X_test.iloc[0:1,:])

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
param_grid = {
    'n_estimators':[50, 200] , 
    'max_samples': [0.6, 0.8]
}

In [ ]:
gscv = GridSearchCV(RandomForestClassifier(max_depth = 4,
                                           oob_score=True,
                                           class_weight='balanced',
                                           random_state=7), 
                    param_grid, cv=2, verbose=2)

In [ ]:
gscv.fit(X_train_pca,y_train)

In [ ]:
gscv.best_score_

In [ ]:
gscv.best_params_

In [ ]:
### Create RF  model using best params

In [ ]:
rfc = RandomForestClassifier(n_estimators = 200,
                       max_samples= 0.6,
                        max_depth = 4,
                        oob_score=True,
                        class_weight='balanced',
                        random_state=7)

In [ ]:
rfc.fit(X_train_pca,y_train)

In [ ]:
y_pred_train = rfc.predict(X_train_pca)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train, y_pred_train)

In [ ]:
# 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {
    "C" : [0.1, 2],
    "gamma" : [0.5, 1]
}

gscv = GridSearchCV(SVC(random_state=7), param_grid, cv=2, verbose=2)


gscv.fit(X_train_pca,y_train)
gscv.best_score_
gscv.best_params_

# Simple logistic regression

In [ ]:
import pandas as pd
df = pd.read_csv("BankChurners_cleaned.csv")

X = df.drop("Attrition_Flag",axis = 1)
Y = df["Attrition_Flag"]

X_ohe = pd.get_dummies(X)

Y.value_counts()

Y.loc[Y=='Existing Customer'] = 0
Y.loc[Y=='Attrited Customer'] = 1

Y.value_counts()
Y.dtype
Y = Y.astype("int")
Y.dtype

# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_ohe,Y,test_size = 0.3 ,random_state = 7,stratify = Y)

# Apply Logistic regression / logit classifier
###### class sklearn.linear_model.LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)[source]¶

#n_jobs = if you want to run multiclass classifier, or how many threads you want to run
#l1_ratio

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty = 'elasticnet',class_weight = 'balanced',random_state = 7, solver = 'saga',max_iter = 100,l1_ratio = 0.5)
# elasticnet = 0.5l1 +0.5l2

lr.fit(X_train,Y_train)

# convergenceWarning = loss is not minimum, till now the best boundary is not found
# model convergence = convergence = means achieving optimial soln. here achieving minimum loss.

Y_pred = lr.predict(X_test)

# Evaluation
## confusion matrix
### sklearn.metrics.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)

from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test,Y_pred)

# true negative false positive
# false negative true positive

# true positive is 0 becuse our model is not converged yet.





# End to End classification

In [ ]:
import pandas as pd

# Check The Data given by Competetion
#training data : all columns
#Consider training data as full data and train test split on it
#test data : target columns missing : can't be used to evaluate our model

train_df = pd.read_csv("train_jRxnrHD.csv")
train_df.info()
test_df = pd.read_csv("test_QaJU1Mh.csv")
test_df.info()

# Split the data

data = train_df.copy()

## Here target is in column 12 'target'
#Premium is another target column which can be predicted for regression problem / multi-class problem

X = data.drop(['premium', 'target'], axis=1)
y = data.loc[:,'target']

X.shape, y.shape

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,
                 test_size=0.3,
                 random_state=7,
                 stratify=y)

X_train.shape,X_test.shape,y_train.shape,y_test.shape

y.value_counts(normalize=True)
y_train.value_counts(normalize=True)
y_test.value_counts(normalize=True)

# Preprocessing

#- Remove the id columns or columns with unique values 
#- here remove id column
#- Remove columns which contain ONLY one value
#- Remove columns / rows containing NAs more than threshold
#- Fill missing values with median
#- Encode the string columns


## Remove the id columns or columns with unique values 

drop_columns = ['id']
X_train.drop(drop_columns,axis=1,inplace=True)

## columns which contain ONLY one value

#NO columns 

X_train.nunique()

## columns / rows containing NAs more than threshold

#NO columns

X_train.isna().sum()/ X_train.shape[0]

## Fill missing values with median

X_train.median()
X_train.mode() #it will be used if na values are there in string type columns

fill_value = X_train.median()
X_train.fillna(fill_value, inplace=True)

# Encode the string columns

#- ONE hot encoding

X_train.select_dtypes(include='object').columns
X_train = pd.get_dummies(X_train)
X_train.shape
X_train.columns

## Standardization

##ONLY on numeric columns and not for categorical columns

cat_columns = ['sourcing_channel', 'residence_area_type']
num_columns = ['perc_premium_paid_by_cash_credit', 'age_in_days', 'Income',
       'Count_3-6_months_late', 'Count_6-12_months_late',
       'Count_more_than_12_months_late', 'application_underwriting_score',
       'no_of_premiums_paid']

from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train.loc[:, num_columns])

### Method 1 (Try to avoid)

trans_data = std.transform(X_train.loc[:, num_columns])
X_train_trans = pd.DataFrame(trans_data,\
                             columns = num_columns,\
                             index = X_train.index)

ohe_columns = ['sourcing_channel_A', 'sourcing_channel_B',
       'sourcing_channel_C', 'sourcing_channel_D', 'sourcing_channel_E',
       'residence_area_type_Rural', 'residence_area_type_Urban']

cat_df = X_train.loc[:, ['sourcing_channel_A', 'sourcing_channel_B',
       'sourcing_channel_C', 'sourcing_channel_D', 'sourcing_channel_E',
       'residence_area_type_Rural', 'residence_area_type_Urban']]

X_train_final = pd.concat([X_train_trans, cat_df], axis=1)
X_train_final.shape, X_train.shape

### Another syntax  (EASY) ###########

X_train_sc = X_train.copy()
std.fit(X_train_sc.loc[:, num_columns])
X_train_sc.loc[:, num_columns] = std.transform(X_train_sc.loc[:, num_columns])
X_train_sc.shape, X_train.shape



# Outlier Handling
#- impute the outliers in numeric columns using IQR method

#IQR = Q3 - Q1
#min_val_val = Q1 - 1.5 IQR
#max_val_val = Q3 + 1.5 IQR

def impute_IQR(df, col):
    q1,q3 = df[col].quantile([0.25, 0.75])
    iqr = q3-q1
    min_val_val = q1 - 1.5 * iqr
    max_val_val = q3 + 1.5 * iqr
    df.loc[df[col] < min_val_val , col] = min_val_val
    df.loc[df[col] > max_val_val , col] = max_val_val
    return df

for col in num_columns:
    X_train_sc = impute_IQR(X_train_sc,col)

X_train_sc.describe()

# Feature Selection

#- PCA
#- RFE
#- SelectFromModel ( Decision Tree)

## PCA

from sklearn.decomposition import PCA
pca = PCA(n_components = 0.99, random_state=7)
pca.fit(X_train_sc)
X_train_pca = pca.transform(X_train_sc)
X_train_pca.shape

# Train the model

#- SVM
#fine tune C and gamma parameters
#- RandomForest
# n_estimators , max_samples , max_depth needs to be tuned
#- CatBoost

## SVM classifier ( SVC)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {
    "C" : [0.1, 2],
    "gamma" : [0.5, 1]
}

gscv = GridSearchCV(SVC(random_state=7), param_grid, cv=2, verbose=2)
gscv.fit(X_train_pca,y_train)
gscv.best_score_
gscv.best_params_

## RandomForest 

from sklearn.ensemble import RandomForestClassifier
param_grid = {
    'n_estimators':[50, 200] , 
    'max_samples': [0.6, 0.8]
}

gscv = GridSearchCV(RandomForestClassifier(max_depth = 4,
                                           oob_score=True,
                                           class_weight='balanced',
                                           random_state=7), 
                    param_grid, cv=2, verbose=2)

gscv.fit(X_train_pca,y_train)

gscv.best_score_
gscv.best_params_

### Create RF  model using best params

rfc = RandomForestClassifier(n_estimators = 200,
                       max_samples= 0.6,
                        max_depth = 4,
                        oob_score=True,
                        class_weight='balanced',
                        random_state=7)

rfc.fit(X_train_pca,y_train)

### predict on train data only because test data is not preprocessed

y_pred_train = rfc.predict(X_train_pca)

from sklearn.metrics import f1_score
f1_score(y_train, y_pred_train)

# Make Test Data Ready fore predictions
#same steps of preprocessing as that of training data are done 
#same features are selected here


X_test.drop(drop_columns, axis=1,inplace=True)
X_test.fillna(fill_value, inplace=True)
X_test_ohe = pd.get_dummies(X_test)
#X_test_ohe.shape, X_test.shape
X_test_ohe.loc[:, num_columns] = std.transform(X_test_ohe.loc[:, num_columns])
#X_test_ohe.shape
X_test_ohe.describe()

### feature selection using PCA for test data

X_test_pca = pca.transform(X_test_ohe)
#X_test_pca.shape

# prediction on Test data

y_pred = rfc.predict(X_test_pca)

f1_score(y_test,y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

# Apply the model to predict on new data
#- apply all the preprocessing steps
#- apply feature selection steps


rfc.predict(X_test_pca[:1])
rfc.predict_proba(X_test_pca[:1])

